# Trabalho final da disciplina SCDIII
Hermes Gomes Araújo
1931133054

In [ ]:
#Imports

import numpy as np
import pandas as pd
from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
# Subindo o banco
base = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv', parse_dates=['Last Update'])

In [ ]:
# Filtrando somente por Brasil
df = base[base['Country/Region'] == 'Brazil']

In [ ]:
#Criando o campo contaminados
df['Contamined'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

In [ ]:
# Observando os dados
df

Notamos vários dados com última atualização em 08/03/2020, mas em dias completamente diferentes, vamos consertar isso

In [ ]:
#Consertando a coluna
df[df['Last Update'] == '2020-03-08 05:31:00']

In [ ]:
#Consertando as linhas
df.loc[7637,'Last Update'] = datetime.datetime(2020, 3, 22)
df.loc[9443,'Last Update'] = datetime.datetime(2020, 3, 28)
df.loc[9754,'Last Update'] = datetime.datetime(2020, 3, 29)
df.loc[10066,'Last Update'] = datetime.datetime(2020, 3, 30)

In [ ]:
#Reindexando o dataframe
df.reset_index(drop=True, inplace=True)

## Iniciando o modelo ARIMA

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.arima_model import ARIMA

# Criando o modelo
model = ARIMA(df.Contamined, order=(1,1,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())

Vemos que com essas configurações, o p-valor de todas as partes do modelo, na quarta coluna, é menor do que 0,05 (um nivel de significância amplamente utilizado). Isso significa que a constante, a parte AR e as partes MA são significativas para o modelo apresentado.

In [ ]:
# Plotando os erros
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':120})

residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Resíduos", ax=ax[0])
residuals.plot(kind='kde', title='Densidade', ax=ax[1])
plt.show()

Idealmente teríamos uma média próxima de 0 o que de certa forma acontece, mas teríamos uma variação constante também. Entretanto a base tem poucos registros e a atualização dos recuperados aconteceu de uma vez. apresentando grande queda no número de contaminados. Dessa Forma, a análise de resíduos é tão boa quanto poderia ser.

In [ ]:
# Realidade vs Modelo
model_fit.plot_predict(dynamic=False)
plt.show()

Vemos que a diferença da realidade para o modelo não é tão diferente assim. Em seguinda vamos tentar predizer para os próximos dias o comportamento.

In [ ]:
!pip install pmdarima

In [ ]:
#Achando a melhor configuração do modelo com stepwise
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

model = pm.auto_arima(df.Contamined, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
#Prevendo os próximos dias

# Até o fim do mês
n_periods = 11
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(df.Contamined), len(df.Contamined)+n_periods)

# Series para gerar o gráfico
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Gráfico
plt.plot(df.Contamined)
plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Previsão de contaminados para o resto de Abril/2020")
plt.show()

Vemos então que até o fim do abril, segundo o modelo que geramos, o range da previsão de contaminados estará entre 20 e 45 mil aproximadamente. O valor previsto um pouco maior do que 30 mil. 